# Generating pitch tracks for the Guše visualizer

In [125]:
import json
import os
import numpy as np
import os
import matplotlib.pyplot as plt

In [126]:
folderName = '../files/pitchTracks/'
mbid = '0430401f-6ca3-404c-b4e2-458664ee2b33'
pitchTrackExtension = '.f0_fil.csv'
fileName = mbid + pitchTrackExtension

dur = 71
shahed = 1045.32

filePath = os.path.join(folderName, fileName)
print(filePath)

../files/pitchTracks/0430401f-6ca3-404c-b4e2-458664ee2b33.f0_fil.csv


### From Hz to cents

In [127]:
def h2c(t, h):
    return np.rint(1200 * np.log2(h/t))

In [128]:
# Load csv
pitchTrackHz = np.genfromtxt(filePath, delimiter=",")
pitchTrackHz

array([[4.400000e-01, 7.849840e+02],
       [4.500000e-01, 7.837230e+02],
       [4.600000e-01, 7.820890e+02],
       ...,
       [7.073000e+01, 1.043558e+03],
       [7.074000e+01, 1.041850e+03],
       [7.075000e+01, 1.037719e+03]])

In [129]:
maxCent = h2c(shahed, np.max(pitchTrackHz[:,1]))
print("Max:", maxCent)

minCent = h2c(shahed, np.min(pitchTrackHz[:,1]))
print("Min:", minCent)

Max: 548.0
Min: -722.0


In [130]:
maxCent = 548
minCent = -722

minCut = True

In [131]:
# Prepare dic
pitchTrackDic = {}

for frame in pitchTrackHz:
    pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]

In [132]:
# Find duration

if pitchTrackHz[-1,0] > dur:
    dur = pitchTrackHz[-1,0]

dur

71

In [133]:
# Cent pitch track

pitchTrackCents = {}

for i in range(dur * 100):
    k = "{:.2f}".format(i/100)
    if k not in pitchTrackDic.keys():
        pitchTrackCents[k] = "s"
    else:
        c = h2c(shahed, pitchTrackDic[k])
        if minCut:
            if c >= minCent and c <= maxCent:
                pitchTrackCents[k] = c
        else:
            pitchTrackCents[k] = c

pitchTrackCents

{'0.00': 's',
 '0.01': 's',
 '0.02': 's',
 '0.03': 's',
 '0.04': 's',
 '0.05': 's',
 '0.06': 's',
 '0.07': 's',
 '0.08': 's',
 '0.09': 's',
 '0.10': 's',
 '0.11': 's',
 '0.12': 's',
 '0.13': 's',
 '0.14': 's',
 '0.15': 's',
 '0.16': 's',
 '0.17': 's',
 '0.18': 's',
 '0.19': 's',
 '0.20': 's',
 '0.21': 's',
 '0.22': 's',
 '0.23': 's',
 '0.24': 's',
 '0.25': 's',
 '0.26': 's',
 '0.27': 's',
 '0.28': 's',
 '0.29': 's',
 '0.30': 's',
 '0.31': 's',
 '0.32': 's',
 '0.33': 's',
 '0.34': 's',
 '0.35': 's',
 '0.36': 's',
 '0.37': 's',
 '0.38': 's',
 '0.39': 's',
 '0.40': 's',
 '0.41': 's',
 '0.42': 's',
 '0.43': 's',
 '0.44': -496.0,
 '0.45': -499.0,
 '0.46': -502.0,
 '0.47': -504.0,
 '0.48': -501.0,
 '0.49': -496.0,
 '0.50': -492.0,
 '0.51': -490.0,
 '0.52': -491.0,
 '0.53': -491.0,
 '0.54': -493.0,
 '0.55': -495.0,
 '0.56': -496.0,
 '0.57': -490.0,
 '0.58': -482.0,
 '0.59': -484.0,
 '0.60': -489.0,
 '0.61': -491.0,
 '0.62': -489.0,
 '0.63': -490.0,
 '0.64': -491.0,
 '0.65': -496.0,
 '0.66': -

In [134]:
with open(mbid + "_pitchTrack.json", 'w') as f:
    json.dump(pitchTrackCents, f)

In [135]:
x = 0
y = 0
for k in pitchTrackCents.keys():
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] > x:
        x = pitchTrackCents[k]
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] < y:
        y = pitchTrackCents[k]
print(y, x)

-722.0 548.0


### Pitch histogram

In [136]:
import intonation
import numpy as np
import json
import os

In [137]:
def pitch2cents(pitch, ref):
    '''
    Converts a pitch track in Hz to cents given a reference pitch
    
    Args:
        pitch (np.array): a numpy array with two columns, first for time,
                          second for pitch
        ref (float): the reference pitch
    
    Returns:
        cents (np.array) a numpy array with one column of pitch in cents
    '''

    return 1200 * np.log2(pitch[:,1] / float(ref))

In [138]:
# folderName = './'
# mbid = '7d88c651-5dea-48e5-b21a-ac964cff5cad'
# pitchTrackExtension = '.f0_fil.csv'

# shahed = 332.67

In [139]:
pitchTrack = np.genfromtxt(os.path.join(folderName, mbid + pitchTrackExtension), delimiter=',')
centsTrack = pitch2cents(pitchTrack, shahed)
filteredTrack = centsTrack[np.logical_and(centsTrack >= minCent, centsTrack <= maxCent)]
pitch_obj = intonation.Pitch(np.arange(len(filteredTrack)), filteredTrack)
# pitch_obj = intonation.Pitch(np.arange(len(centsTrack)), centsTrack)
rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist()

print(rec_obj.histogram.x)
print(rec_obj.histogram.y)

[-716.19198279 -715.19175512 -714.19152745 ...  545.09510849  546.09533616
  547.09556383]
[2.14249567e-05 2.11101563e-05 2.05067957e-05 ... 3.73251142e-05
 3.74239187e-05 3.74901808e-05]


In [140]:
histDic = {}

for i in range(rec_obj.histogram.x.size):
    # pos = '{:.2f}'.format(rec_obj.histogram.x[i])
    x = round(rec_obj.histogram.x[i], 2)
    y = round(rec_obj.histogram.y[i] * 10000, 2)
    histDic[x] = y

histMin = np.min(list(histDic.values()))
histMax = np.max(list(histDic.values()))

print(histMin, histMax)

histList = []
for k, v in histDic.items():
    histList.append([k,v])

hist2json = {'min': histMin, 'max': histMax, 'hist': histList}
hist2json

0.0 130.82


{'min': 0.0,
 'max': 130.82,
 'hist': [[-716.19, 0.21],
  [-715.19, 0.21],
  [-714.19, 0.21],
  [-713.19, 0.2],
  [-712.19, 0.19],
  [-711.19, 0.18],
  [-710.19, 0.16],
  [-709.19, 0.15],
  [-708.19, 0.14],
  [-707.19, 0.14],
  [-706.19, 0.13],
  [-705.19, 0.13],
  [-704.19, 0.12],
  [-703.19, 0.12],
  [-702.19, 0.12],
  [-701.19, 0.12],
  [-700.19, 0.12],
  [-699.19, 0.13],
  [-698.19, 0.13],
  [-697.19, 0.13],
  [-696.19, 0.13],
  [-695.19, 0.14],
  [-694.19, 0.14],
  [-693.19, 0.15],
  [-692.19, 0.17],
  [-691.19, 0.19],
  [-690.19, 0.21],
  [-689.19, 0.23],
  [-688.19, 0.26],
  [-687.19, 0.29],
  [-686.19, 0.33],
  [-685.18, 0.36],
  [-684.18, 0.4],
  [-683.18, 0.44],
  [-682.18, 0.47],
  [-681.18, 0.49],
  [-680.18, 0.52],
  [-679.18, 0.53],
  [-678.18, 0.54],
  [-677.18, 0.54],
  [-676.18, 0.53],
  [-675.18, 0.51],
  [-674.18, 0.49],
  [-673.18, 0.46],
  [-672.18, 0.43],
  [-671.18, 0.39],
  [-670.18, 0.35],
  [-669.18, 0.31],
  [-668.18, 0.27],
  [-667.18, 0.23],
  [-666.18, 0.1

In [141]:
pitchHistogramFolder = './'
pitchHistogramExtension = '_pitchHistogram.json'

json.dumps(hist2json)

with open(os.path.join(pitchHistogramFolder, mbid + pitchHistogramExtension), 'w') as f:
    json.dump(hist2json, f)